In [2]:
import sys
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset

sys.path.append('NFT/')
from dicts import data_to_num_vars_dict

sys.path.append('NFT/data/')
from proccess_data_functions import get_datasets, plot_data

sys.path.append('NFT/')
from training_functions import read_all_data_and_print_stats, train_model, evaluate_model, save_model

sys.path.append('NFT/models/baseline_models/')
from base_models import FC, MV_LSTM, CNN, TCN, TimeSeriesTransformer

sys.path.append('NFT/models/NFT/')
from NFT import NFT

data = 'air_quality'

NUM_OF_VARS = data_to_num_vars_dict[data]
LOOKBACK = 15
HORIZON = 5

BATCH_SIZE = 32
NUM_EPOCHS = 5
PLOT_EPOCH = NUM_EPOCHS

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

n_people = 600
n_rows = 700
    
class dataset(Dataset):
    def __init__(self, data, target):
        self.input = data
        self.target = target

    def __len__(self):
        return len(self.input)

    def __getitem__(self, idx): 
        return self.input[idx], self.target[idx]


In [ ]:
data_path = f"NFT/data/{data}/{data}_{LOOKBACK}l_{HORIZON}h/"

train_X, train_y, val_X, val_y, test_X, test_y = read_all_data_and_print_stats(data_path=data_path)


In [9]:
NUM_EPOCHS = 3


In [10]:
model = NFT(
    device=device,
    forecast_length=HORIZON,
    backcast_length=LOOKBACK,
    n_vars=NUM_OF_VARS,
    nb_blocks_per_stack=2,
    num_channels_for_tcn=[25, 50]
    ).to(device)

model.compile(loss='mse', optimizer='adam')

model.fit(
    x_train=train_X, 
    y_train=train_y, 
    validation_data=(val_X, val_y), 
    epochs=NUM_EPOCHS,
    batch_size=BATCH_SIZE, 
    plot_epoch=PLOT_EPOCH
    )

evaluate_model(model, train_X, train_y, val_X, val_y, test_X, test_y)


device = cuda
| N-Beats
| --  Stack Trend (#0) (share_weights_in_stack=False)
     | -- TrendBlock(units=256, thetas_dim=4, backcast_length=15, forecast_length=5, share_thetas=True) at @140172959291808
     | -- TrendBlock(units=256, thetas_dim=4, backcast_length=15, forecast_length=5, share_thetas=True) at @140172917861488
| --  Stack Seasonality (#1) (share_weights_in_stack=False)
     | -- SeasonalityBlock(units=256, thetas_dim=8, backcast_length=15, forecast_length=5, share_thetas=True) at @140172925864640
     | -- SeasonalityBlock(units=256, thetas_dim=8, backcast_length=15, forecast_length=5, share_thetas=True) at @140172917862352
Epoch 1, Average loss per data point: 0.053742938794253764
Epoch 1/3
375/375 [==============================] - 3s 10ms/step - loss: 0.0537 - val_loss: 0.0006
Epoch 2, Average loss per data point: 7.855969550698694e-05
Epoch 2/3
375/375 [==============================] - 3s 9ms/step - loss: 0.0001 - val_loss: 0.0000
Epoch 3, Average loss per data point

The min train loss is 7.855969550698694e-05 at epoch 1
The min val loss is 1.310487186856335e-05 at epoch 1

MSE:
train MSE = 0.00031818822026252747
val MSE = 0.00031847122590988874
test MSE = 0.00031847122590988874

sMAPE:
train sMAPE = 8.156152069568634
val sMAPE = 8.131808042526245
test sMAPE = 8.131808042526245

MAPE:
train MAPE = 6.552122533321381
val MAPE = 6.5380677580833435
test MAPE = 6.5380677580833435

MASE:
train MAPE = 0.5315811038017273
val MAPE = 0.532177746295929
test MAPE = 0.532177746295929



(tensor(0.0003, device='cuda:0'),
 tensor(0.0003, device='cuda:0'),
 8.156152069568634,
 8.131808042526245,
 6.552122533321381,
 6.5380677580833435,
 0.5315811,
 0.53217775)

In [12]:
tcn_model = TCN(lookback=LOOKBACK, 
                horizon=HORIZON, 
                num_vars=NUM_OF_VARS, 
                device=device,
                num_channels=[25, 50], 
                kernel_size=2, 
                dropout=0.2).to(device)

train_model(dataset=dataset,
            epochs=NUM_EPOCHS,
            model=tcn_model, 
            train_X=train_X.to(device), train_y=train_y.to(device),
            val_X=val_X.to(device), val_y=val_y.to(device), 
            device=device, 
            lookback=LOOKBACK,
            horizon=HORIZON, 
            n_vars=NUM_OF_VARS, 
            batch_size=BATCH_SIZE, 
            print_epoch=PLOT_EPOCH, 
            path_to_save_prediction_plots=None
            )

evaluate_model(tcn_model, train_X, train_y, val_X, val_y, test_X, test_y)

Epoch 1/3, loss: 0.0822 - val_loss: 0.0005
Epoch 2/3, loss: 0.0015 - val_loss: 0.0001


In [3]:
total_cycles=200
points_per_period=100

NUM_OF_VARS = 4

def generate_seasonal_data(total_cycles, points_per_period):
    x = np.arange(total_cycles * points_per_period)
    y = np.sin(2 * np.pi * x / points_per_period)
    return y

def plot_data(data):
    plt.figure(figsize=(10, 4))
    plt.plot(data)
    plt.title("Synthetic Seasonal Time Series Data")
    plt.xlabel("Time")
    plt.ylabel("Value")
    plt.grid(True)
    plt.show()

def create_multivar_series(total_cycles, points_per_period):
    d = []
    for _ in range(NUM_OF_VARS):
        d.append(generate_seasonal_data(total_cycles, points_per_period))
    multivariate_series = np.column_stack(d)
    print(multivariate_series.shape, multivariate_series.dtype)
    return torch.FloatTensor(multivariate_series)

def create_data():
    data = create_multivar_series(total_cycles, points_per_period)

    train_size = int(len(data) * 0.6)
    test_size = int(len(data) * 0.2)
    val_size = len(data) - train_size - test_size

    train = data[:train_size]
    val = data[train_size:train_size + val_size]
    test = data[train_size + val_size:train_size + val_size + test_size]

    train_dataset, val_dataset, test_dataset = get_datasets(train, val, test, LOOKBACK, HORIZON)

    train_X, train_y = train_dataset.X, train_dataset.y
    val_X, val_y = val_dataset.X, val_dataset.y
    test_X, test_y = test_dataset.X, test_dataset.y
    
    return train_X, train_y, val_X, val_y, test_X, test_y

train_X, train_y, val_X, val_y, test_X, test_y = create_data()

print(train_X.dtype, train_y.dtype, val_X.dtype)

print(f"shape of:\n"
f"train: X {train_X.shape}, y: {train_y.shape}\n"
f"val: X {val_X.shape}, y: {val_y.shape}\n"
f"test: X {test_X.shape}, y: {test_y.shape}")


(20000, 4) float64
torch.float32 torch.float32 torch.float32
shape of:
train: X torch.Size([11980, 15, 4]), y: torch.Size([11980, 5, 4])
val: X torch.Size([3980, 15, 4]), y: torch.Size([3980, 5, 4])
test: X torch.Size([3980, 15, 4]), y: torch.Size([3980, 5, 4])
